In [73]:
%matplotlib inline
from qiskit import QuantumCircuit, Aer
from qiskit.compiler import transpile
from qiskit.visualization import *

import math
from math import pi


In [74]:
#each of these functions replaces the given gate with a combination of rx, rz and cz, and then returns the overhead increase

def convertid(qc,i):
    return 0

def converth(qc,i):
    qc.rz(0.5*pi,i)
    qc.rx(0.5*pi,i)
    qc.rz(0.5*pi,i)
    return 2

def convertx(qc,i):
    qc.rx(pi,i)
    return 0

def converty(qc,i):
    qc.rx(pi,i)
    qc.rz(pi,i)
    return 1

def convertz(qc,i):
    qc.rz(pi,i)
    return 0
    
def convertrx(qc,i,theta):
    qc.rx(theta,i)
    return 0

def convertry(qc,i,theta):
    qc.rz(0.5*pi,i)
    qc.rx(theta,i)
    qc.rz((-0.5)*pi,i)
    return 2

def convertrz(qc,i,theta):
    qc.rz(theta,i)
    return 0

def convertcz(qc,cq,tq):
    qc.cz(cq,tq)
    return 0

In [75]:
def transform(qc_ip, n):
    
    overhead = 0
    
    qc_op = QuantumCircuit(n)
    
    for i in qc_ip.data:
        if i[0].name == 'id':
            overhead += convertid(qc_op, i[1][0].index)
        
        elif i[0].name == 'h':
            overhead += converth(qc_op, i[1][0].index)
        
        elif i[0].name == 'x':
            overhead += convertx(qc_op, i[1][0].index)
        
        elif i[0].name == 'y':
            overhead += converty(qc_op, i[1][0].index)
            
        elif i[0].name == 'z':
            overhead += convertz(qc_op, i[1][0].index)
        
        elif i[0].name == 'rx':
            overhead += convertrx(qc_op, i[1][0].index, i[0].params[0])
            
        elif i[0].name == 'ry':
            overhead += convertry(qc_op, i[1][0].index, i[0].params[0])
                
        elif i[0].name == 'rz':
            overhead += convertrz(qc_op, i[1][0].index, i[0].params[0])    
            
        elif i[0].name == 'cz':
            overhead += convertcz(qc_op, i[1][0].index, i[1][1].index)
        
        else:
            print("Gate cannot be processed")
            return
        
    return (qc_op, overhead)

In [76]:
qc = QuantumCircuit(4)
qc.x(0)
qc.h(1)
qc.cz(1,2)
qc.y(3)
qc.ry(pi/3.0,3)
qc.y(2)

print(qc)

     ┌───┐                 
q_0: ┤ X ├─────────────────
     ├───┤                 
q_1: ┤ H ├─────■───────────
     └───┘     │      ┌───┐
q_2: ──────────■──────┤ Y ├
     ┌───┐┌──────────┐└───┘
q_3: ┤ Y ├┤ RY(pi/3) ├─────
     └───┘└──────────┘     


In [77]:
qc_new, overhead = transform(qc, 4)

print(qc_new)
print("Overhead :", overhead)

      ┌────────┐                                                            
q_0: ─┤ RX(pi) ├────────────────────────────────────────────────────────────
     ┌┴────────┴┐┌──────────┐┌──────────┐                                   
q_1: ┤ RZ(pi/2) ├┤ RX(pi/2) ├┤ RZ(pi/2) ├─────■─────────────────────────────
     └──────────┘└──────────┘└──────────┘     │        ┌────────┐ ┌────────┐
q_2: ─────────────────────────────────────────■────────┤ RX(pi) ├─┤ RZ(pi) ├
      ┌────────┐  ┌────────┐ ┌──────────┐┌──────────┐┌─┴────────┴┐└────────┘
q_3: ─┤ RX(pi) ├──┤ RZ(pi) ├─┤ RZ(pi/2) ├┤ RX(pi/3) ├┤ RZ(-pi/2) ├──────────
      └────────┘  └────────┘ └──────────┘└──────────┘└───────────┘          
Overhead : 6


ImportError: The class MatplotlibDrawer needs pylatexenc. to install, run "pip install pylatexenc".